# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive,spark
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")



# Preparation

In [3]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [4]:
date_params

{'mediawiki_history_snapshot': '2021-10',
 'metrics_month': '2021-10',
 'metrics_month_first_day': '2021-10-01',
 'metrics_month_last_day': '2021-10-31',
 'metrics_year': 2021,
 'metrics_cur_month': 10}

# MariaDB and Hive query metrics


In [5]:
queries = {
    "structured_data_used": {
        "file": "queries/structured_data_used.hql",
        "engine": "hive"
    },
    
    "num_commons_files_used_content_pages": {
        "file": "queries/num_commons_files.hql",
        "engine": "hive"
    },
    "wikidata_items": {
        "file": "queries/wikidata_items.hql",
        "engine": "hive"
    },
    "wikidata_items_being_reused": {
        "file": "queries/wikidata_items_being_reused.hql",
        "engine": "hive"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "hive":
        result = hive.run(query)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running structured_data_used on hive...
Running num_commons_files_used_content_pages on hive...
Running wikidata_items on hive...
Running wikidata_items_being_reused on hive...


In [6]:
result

,month,wikidata_items_being_reused
0,2021-10-01,19801319


# Combining and saving metrics

In [7]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()


In [8]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

,structured_data_used,num_commons_files_used_content_pages,wikidata_items,wikidata_items_being_reused
month,,,,
2021-01-01,110336427.0,30922629.0,91898784.0,18890176.0
2021-02-01,111537756.0,31236223.0,92346206.0,18961132.0
2021-03-01,113749241.0,31994067.0,92766054.0,19081836.0
2021-04-01,115768032.0,32439596.0,93039049.0,19175909.0
2021-05-01,117023506.0,33188319.0,93295754.0,19272296.0
2021-06-01,117956153.0,33484828.0,93525712.0,19379045.0
2021-07-01,118873343.0,33853754.0,93808863.0,19464104.0
2021-08-01,120411293.0,35339547.0,94347261.0,19540328.0
2021-09-01,121413643.0,35748687.0,94646283.0,19640044.0


In [9]:
metrics.to_csv(FILENAME, sep="\t")